# Hello docker (for Data Scientists & Developers)

source : https://docs.docker.com/get-started/

---
### Docker 개념과 장점

Docker 컨테이너는 독립적으로 애플리케이션이나 프로세스를 실행할 수 있는 경량화된 컴퓨팅환경입니다. Docker를 이용할 때 다음과 같은 장점을 가질 수 있으며 이런 점 때문에 최근 가장 주목받는 컴퓨팅 실행환경으로 선택받고 있습니다.

- 유연성(Flexible): Even the most complex applications can be containerized.
- 경량(Lightweight): Containers leverage and share the host kernel, making them much more efficient in terms of system resources than virtual machines.
- 이식성(Portable): You can build locally, deploy to the cloud, and run anywhere.
- 느슨한 결합(Loosely coupled): Containers are highly self sufficient and encapsulated, allowing you to replace or upgrade one without disrupting others.
- 확장성(Scalable): You can increase and automatically distribute container replicas across a datacenter.
- 보안(Secure): Containers apply aggressive constraints and isolations to processes without any configuration required on the part of the user.

이런 장점들은 특히 머신러닝의 실행환경에서 더욱 가치를 발휘합니다. 왜냐하면,

- 머신러닝 코드에서 사용하게 되는 복잡한 dependency 관계를 자연스럽게 코드로 정의하게 되며, 동일한 실행환경을 언제든 반복적으로 재생성할 수 있습니다.
- 일반적으로 학습단계에 대규묘 병렬 컴퓨팅을 필요로 하지만 그 필요량과 시점을 예측하기 어렵습니다. 머신러닝과 같이 동적으로 리소스를 배정해야 하는 환경에서 보다 효율적으로 리소스를 관리할 수 있습니다. 
- 머신러닝의 배포단계에서 추가 작업량이 현저히 줄어들게 됩니다. 머신러닝의 응용환경 적용대상은 클라우드 서버에서부터 모바일, IoT Edge에 이르기까지 다양합니다. 이런 다양한 환경에 모델을 배포할 때 환경설정으로 위한 시행착오를 줄일 수 있습니다.
- 자동화된 재학습과 자동화된 배포구성시 더욱 용이하게 구성할 수 있습니다.

### Docker 구조

![](https://docs.docker.com/engine/images/architecture.svg)


### Docker 기본 명령

Docker는 많은 복잡하고 다양한 명령과 설정을 포함하고 있지만 인프라 운영자가 아닌 Data Scientist 또는 Developer의 관점에서 알아야 할 Docker 명령은 그리 많지 않습니다. 본 노트북에서는 예제를 통해 다음 기본 명령들이 어떤 기능을 하는지 이해하고 활용할 수 있는 것을 목적으로 합니다.

- docker build
- docker run
- docker pull/push
- docker image
- docker ps

---
### Docker 환경 점검

SageMaker 노트북 환경에는 이미 도커가 설치되어 있습니다.

In [1]:
!docker --version

`docker images` 명령을 통해 docker image repository를 확인할 수 있습니다.

In [6]:
!docker images

`docker ps` 명령을 통해 컨터이너 실행환경의 프로세스를 확인할 수 있습니다.

In [5]:
!docker ps -a

다음 셀의 명령은 도커 레지스트리에서 아래 이미지를 로컬로 복사하여 가져옵니다. 
- https://hub.docker.com/_/busybox

In [39]:
!docker pull busybox

In [40]:
!docker images

In [42]:
!docker run busybox echo "hello docker!!"

In [44]:
!docker ps -a

---
### Docker build 

Dcokerfile을 이용하여 실행환경을 정의합니다.
- ubuntu 18.04 를 base image로 사용합니다.
- wget, python, nginx 등 추가 필요한 필요한 도구와 서비스를 정의합니다.
- 사용자 프로그램에서 실행할 dependency library를 정의합니다.
- 환경변수와 working directory 등을 구성합니다.
- 맨 아래줄에 사용자 정의 테스트 프로그램 파일을 복사하고 있습니다.

In [29]:
%%writefile Dockerfile
# Use the official image as a parent image.
FROM ubuntu:18.04

# Install tools and utilities 
RUN apt-get -y update && apt-get install -y --no-install-recommends \
     wget \
     python3-pip \
     python3-setuptools \
     nginx \
     ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# install python dependencies
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# set env variables    
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set the working directory.
WORKDIR /opt/program

# Copy the file from your host to your current location.
COPY hello_docker.py .

테스트용으로 사용할 간단한 파이썬 실행코드를 생성합니다. (Dockerfile에서 컨테이너 내부로 copy하여 실행할 파일)

In [35]:
%%writefile hello_docker.py
import numpy as np
import pandas as pd

x = [1,2,3]
pd.DataFrame(x)
print(x)
print('pandas library was installed and runs well!')

방금 정의한 Dockerfile을 이용하여 도커이미지를 빌드합니다.

In [36]:
!docker build -t hello_docker .

In [37]:
!docker images

---
### Docker 실행

`docker run` 명령을 이용하여 방금 빌드한 이미지를 실행합니다.
- `hello_docker` 이미지를 실행하면서 `python hello_docker.py`명령을 실행하고 있습니다.

In [38]:
!docker run hello_docker python hello_docker.py

In [34]:
!docker ps -a

---
### AWS ECR 연결

다음 shell script 코드는 AWS ECR에 "hello-docker"라는 이름의 레포지토리를 만들고 접속한 후 조금 전 생성한 이미지를 레포지토리에 push합니다. 

In [60]:
%%sh
# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
image="hello-docker"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"
echo ${fullname}

# 1) If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# 2) login to ecr
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}".dkr.ecr."${region}".amazonaws.com

# 3) docker push
docker push ${fullname}

실행이 완료되면 AWS 콘솔의 [ECR](https://console.aws.amazon.com/ecr/repositories)로 이동하여 생성된 레포지토리와 push된 이미지를 확인합니다. 
